In [184]:
# Import libraries necessary for this project
import numpy as np
import pandas as pd
from sklearn.cross_validation import ShuffleSplit
%matplotlib inline

# Load the Boston housing dataset
data = pd.read_csv('train.csv')
test_data = pd.read_csv("test.csv")
prices = data['SalePrice']
features = data.drop('SalePrice', axis = 1)
print "Ames housing dataset has {} data points with {} variables each.".format(*features.shape)

Ames housing dataset has 1460 data points with 80 variables each.


In [165]:
# TODO: Minimum price of the data
minimum_price = np.amin(prices)

# TODO: Maximum price of the data
maximum_price = np.amax(prices)

# TODO: Mean price of the data
mean_price = np.mean(prices)

# TODO: Median price of the data
median_price = np.median(prices)

# TODO: Standard deviation of prices of the data
std_price = np.std(prices)

# Show the calculated statistics
print "Statistics for Ames housing dataset:\n"
print "Minimum price: ${:,.2f}".format(minimum_price)
print "Maximum price: ${:,.2f}".format(maximum_price)
print "Mean price: ${:,.2f}".format(mean_price)
print "Median price ${:,.2f}".format(median_price)
print "Standard deviation of prices: ${:,.2f}".format(std_price)

Statistics for Ames housing dataset:

Minimum price: $34,900.00
Maximum price: $755,000.00
Mean price: $180,921.20
Median price $163,000.00
Standard deviation of prices: $79,415.29


In [167]:
#check NAs
NAs = pd.concat([features.isnull().sum(), test_data.isnull().sum()], axis=1, keys=['Train', 'Test'])
NAs[NAs.sum(axis=1) > 10]

,Train,Test
LotFrontage,259,227
Alley,1369,1352
MasVnrType,8,16
MasVnrArea,8,15
BsmtQual,37,44
BsmtCond,37,45
BsmtExposure,38,44
BsmtFinType1,37,42
BsmtFinType2,38,42
FireplaceQu,690,730


In [185]:
#remove features with too many NAs
features = pd.concat([features, test_data],keys=['train', 'test'])
features.drop(['LotFrontage', 'Alley', 'BsmtQual', 'BsmtCond', 'BsmtExposure','BsmtFinType1','BsmtFinType2','FireplaceQu','GarageType','GarageYrBlt','GarageFinish','GarageQual', 'PoolQC', 'Fence', 'MiscFeature'], axis=1, inplace=True)
features.drop(['RoofStyle','RoofMatl','Exterior1st','Exterior2nd','MasVnrType','MasVnrArea','BsmtFinSF1','BsmtFinSF2','WoodDeckSF','3SsnPorch','ScreenPorch','BsmtFullBath','BsmtHalfBath','Functional','GarageCond','LowQualFinSF','MiscVal','GarageCars'],axis=1,inplace=True)
print features.shape
NAs = features.isnull().sum()
print NAs[NAs > 1]
#fill NA
features['MSSubClass'] = features['MSSubClass'].astype(str)
features['MSZoning'] = features['MSZoning'].fillna(features['MSZoning'].mode()[0])
features['Utilities'] = features['Utilities'].fillna(features['Utilities'].mode()[0])


(2919, 47)
MSZoning     4
Utilities    2
dtype: int64


In [186]:
numeric_features = features.loc[:,['LotArea','TotalBsmtSF','1stFlrSF','2ndFlrSF','GrLivArea','OpenPorchSF','EnclosedPorch','GarageArea']]
numeric_features_standardized = (numeric_features - numeric_features.mean())/numeric_features.std()

In [196]:
from sklearn.cross_validation import train_test_split
for col in features.dtypes[features.dtypes == 'object'].index:
    for_dummy = features.pop(col)
    features = pd.concat([features, pd.get_dummies(for_dummy, prefix=col)], axis=1)



ValueError: Found input variables with inconsistent numbers of samples: [2919, 1460]

In [138]:
#feature engineering
from sklearn.decomposition import PCA
pca = PCA(n_components = 4, random_state = 0)
X_train_pca = pca.fit_transform(X_train)
X_valid_pca = pca.fit_transform(X_valid)

### Question 3 - Training and Testing
*What is the benefit to splitting a dataset into some ratio of training and testing subsets for a learning algorithm?*  
**Hint:** What could go wrong with not having a way to test your model?

In [139]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(random_state = 10)
lr.fit(X_train_pca, y_train)
prediction = lr.predict(X_valid_pca)
lr.score(X_valid_pca, y_valid)
#performance_metric(y_valid, prediction)

0.0

**Answer: ** If we train the model with all the data, it will try to match all data points and this can easily lead to overfitting. We need a test set of data to validate the model to be applicable to any new data, not just the known training data set. 

In [140]:
regressor = DecisionTreeRegressor(random_state=0, max_depth = 15)
regressor.fit(X_train_pca, y_train)
prediction = regressor.predict(X_valid_pca)
performance_metric(y_valid, prediction)

0.31136224563756199

In [151]:
# Produce a matrix for client data

sub = pd.DataFrame()
sub['Id'] = test_data['Id']
test_data = pd.get_dummies(test_data)
test = test_data.fillna(test_data.mean())
pca = PCA(n_components = 4, random_state = 0)
test_pca = pca.fit_transform(test)
print test_pca.shape

sub['SalePrice'] = regressor.predict(test_pca)
sub.to_csv("dt_submission.csv",index=False)

(1459, 4)
